In [ ]:
import nest_asyncio
nest_asyncio.apply()

import aiohttp
import asyncio
import json
import pandas as pd
from datetime import datetime 
from datetime import timedelta

# UMS app information
ums_base_url = "https://app.edoofa.com/api/1.1/obj"
ums_bearer_token = "786720e8eb68de7054d1149b56cc04f9"
ums_headers = {'Authorization': f'Bearer {ums_bearer_token}'}





# Asynchronous function to fetch data from a table
async def fetch_table_data(session, base_url, headers, table, constraints=None):
    records = []
    cursor = 0
    total_fetched = 0

    while True:
        params = {'limit': 100, 'cursor': cursor}
        if constraints:
            params['constraints'] = json.dumps(constraints)

        api_url = f"{base_url}/{table}"
        print(f"Fetching {table} data from {base_url}... Cursor: {cursor}")

        async with session.get(api_url, headers=headers, params=params) as response:
            if response.status != 200:
                print(f"Failed to fetch data from {table}: {await response.text()}")
                break

            data = await response.json()
            new_records = data['response']['results']
            records.extend(new_records)
            total_fetched += len(new_records)

            print(f"Fetched {len(new_records)} new records, Total fetched: {total_fetched}")

            cursor += 100

            if len(new_records) < 100:
                print(f"Exiting loop, fetched less than 100 records.")
                break

    df = pd.DataFrame(records)
    print(f"Fetched {len(df)} records for {table}.")
    return df



def get_percentage(number_of_present, total_sessions):
    if total_sessions == 0:
        return 0  # Avoid division by zero
    percent = (number_of_present / total_sessions) * 100
    #print(percent)
    return percent

async def check_if_row_exists(student, year, month, ums_avg_att_df):
    # Filtering the ums_avg_att_df to check if there's an existing record for the student
    existing_rows = ums_avg_att_df[
        (ums_avg_att_df['ewyl-group-name'] == student) &
        (ums_avg_att_df['year'] == year) &
        (ums_avg_att_df['month'] == month)
    ]

    if not existing_rows.empty:
        # Assuming '_id' is the column name for the row ID
        return existing_rows['_id'].iloc[0]
    else:
        return False
    
def process_att_summary(combined_df, ums_avg_att_df):
    att_summary_df = pd.DataFrame(columns=['admissions-group-name', 'attendance-percentage', 
                                           'avg-att-percent-till-last-month', 'ewyl-group-name', 
                                           'first-day-of-month', 'month', 'year'])

    combined_df['date'] = pd.to_datetime(combined_df['date'])    
    unique_students = combined_df['admissions-group-name'].unique()

    for student in unique_students:
        print(f"Processing attendance data for student: {student}")  # Debug line to indicate the current student being processed
        student_df = combined_df[combined_df['admissions-group-name'] == student]

        first_date = pd.to_datetime(student_df['date'].min())
        first_day_current_month = first_date.replace(day=1)

        while True:
            this_month_df = student_df[(student_df['date'].dt.year == first_day_current_month.year) &
                                       (student_df['date'].dt.month == first_day_current_month.month)]

            prev_month_df = student_df[student_df['date'] < first_day_current_month]

            total_sessions_this_month = len(this_month_df)
            attended_sessions_this_month = this_month_df['present'].sum()
            attendance_percentage_this_month = get_percentage(attended_sessions_this_month, total_sessions_this_month)

            if prev_month_df.empty:
                avg_att_till_last_month = 0
            else:
                total_sessions_till_last_month = len(prev_month_df)
                attended_sessions_till_last_month = prev_month_df['present'].sum()
                avg_att_till_last_month = get_percentage(attended_sessions_till_last_month, total_sessions_till_last_month)

            new_row = {
                'admissions-group-name': student,
                'attendance-percentage': attendance_percentage_this_month,
                'avg-att-percent-till-last-month': avg_att_till_last_month,
                'ewyl-group-name': student_df['ewyl-group-name'].iloc[0] if not student_df['ewyl-group-name'].empty else 'N/A',
                'first-day-of-month': first_day_current_month.strftime("%Y-%m-%d"),
                'month': first_day_current_month.strftime("%B"),
                'year': first_day_current_month.year
            }
            att_summary_df = pd.concat([att_summary_df, pd.DataFrame([new_row])], ignore_index=True)

            next_month_first_day = (first_day_current_month + pd.offsets.MonthBegin()).normalize()
            first_day_current_month = next_month_first_day
            
            if student_df[student_df['date'] >= first_day_current_month].empty:
                break

    # Save the processed attendance summary data to a CSV file
    att_summary_df.to_csv('attendance_summary.csv', index=False)
    print("Attendance summary data saved to 'attendance_summary.csv'.")

    return att_summary_df





async def main():

    async with aiohttp.ClientSession() as session:
        ums_att_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance")
        ums_att_df.to_csv("ums_att_df.csv", index=False)
        ums_avg_att_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance-Summary")
        ums_avg_att_df.to_csv("ums_avg_att_df.csv", index=False)
        
        att_summary_df = process_att_summary(ums_att_df, ums_avg_att_df)

        print("Attendance summary processing complete.")
        
await main()


In [22]:
import nest_asyncio
nest_asyncio.apply()

import aiohttp
import asyncio
import json
import pandas as pd
from datetime import datetime 
from datetime import timedelta


student_df = None
engagement_df = None
ums_att_df = None
ums_avg_att_df = None
ums_upload_avg_att=None

edoofy_base_url = "https://edoofa-portal.bubbleapps.io/api/1.1/obj"
edoofy_bearer_token = "2cde31d8f48919a2db1467cc06a56132"
edoofy_headers = {'Authorization': f'Bearer {edoofy_bearer_token}'}

ums_base_url = "https://app.edoofa.com/version-test/api/1.1/obj"
ums_bearer_token = "786720e8eb68de7054d1149b56cc04f9"
ums_headers = {'Authorization': f'Bearer {ums_bearer_token}'}


async def fetch_table_data(session, base_url, headers, table, constraints=None):
    records = []
    cursor = 0
    total_fetched = 0

    while True:
        params = {'limit': 100, 'cursor': cursor}
        if constraints:
            params['constraints'] = json.dumps(constraints)

        api_url = f"{base_url}/{table}"
        print(f"Fetching {table} data from {base_url}... Cursor: {cursor}")

        async with session.get(api_url, headers=headers, params=params) as response:
            if response.status != 200:
                print(f"Failed to fetch data from {table}: {await response.text()}")
                break

            data = await response.json()
            new_records = data['response']['results']
            records.extend(new_records)
            total_fetched += len(new_records)

            print(f"Fetched {len(new_records)} new records, Total fetched: {total_fetched}")

            cursor += 100

            if len(new_records) < 100:
                print(f"Exiting loop, fetched less than 100 records.")
                break

    df = pd.DataFrame(records)
    print(f"Fetched {len(df)} records for {table}.")
    return df



def create_attendance_row(engagement_record):
    attendance_row = {
        'admissions-group-name': engagement_record['admissions-group-name'],  
        'attendance-type': engagement_record['engagement-type'],  
        'date': engagement_record['engagement-date'],  
        'ewyl-group-name': engagement_record['ewyl-group-name'],  
        'present': engagement_record['daily-attendance'],  
    }

    return attendance_row

async def post_attendance_record(session, url, headers, record):
    try:
        async with session.post(url, headers=headers, json=record) as response:
            if response.status == 200:
                print("Record posted successfully.")
                return await response.json()
            else:
                print(f"Failed to post record: {await response.text()}")
                return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


async def main():
    async with aiohttp.ClientSession() as session:
        ums_student_df = await fetch_table_data(session, ums_base_url, ums_headers, "Student")
        ums_att_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance")  
        student_constraints = [{'key': 'indian-edoofian', 'constraint_type': 'equals', 'value': 'yes'}]
        student_df = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Student", constraints=student_constraints)
        
        engagement_df = pd.DataFrame()
        for engagement_type in ['IE Call', 'IE Chat', 'Activity', 'Lesson']:
            engagement_data = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Engagement", [{'key': 'engagement-type', 'constraint_type': 'equals', 'value': engagement_type}])
            engagement_df = pd.concat([engagement_df, engagement_data], ignore_index=True)
        
        engagement_df = pd.merge(engagement_df, student_df[['_id', 'KAM-group-name', 'EWYL-group-name']], left_on='student', right_on='_id', how='left')
        engagement_df.rename(columns={'KAM-group-name': 'admissions-group-name', 'EWYL-group-name': 'ewyl-group-name', '_id_x': '_id'}, inplace=True)
        engagement_df.drop(['_id_y'], axis=1, inplace=True)
        engagement_df.dropna(subset=['admissions-group-name'], inplace=True)
        
        # Convert dates to datetime objects and ensure they are tz-naive
        ums_att_df['date'] = pd.to_datetime(ums_att_df['date']).dt.tz_localize(None)
        engagement_df['engagement-date'] = pd.to_datetime(engagement_df['engagement-date']).dt.tz_localize(None)
        engagement_df['engagement-date'] = pd.to_datetime(engagement_df['engagement-date']) + timedelta(hours=5, minutes=30)
        
        # Create 'comparison-date' in 'ums_att_df' with just the date part
        ums_att_df['date'] = ums_att_df['date'].dt.date
        # Create 'comparison-date' in 'engagement_df' with just the date part
        engagement_df['engagement-date'] = engagement_df['engagement-date'].dt.date
        
        new_attendance_records = []
        
        for _, student in ums_student_df.iterrows():
            admissions_group_name = student['admissions-group-name']
            print(f"Processing admissions group: {admissions_group_name}")

            # Dynamically determine the latest attendance date for each admissions group
            if admissions_group_name in ums_att_df['admissions-group-name'].values:
                group_attendance = ums_att_df[ums_att_df['admissions-group-name'] == admissions_group_name]
                if not group_attendance.empty:
                    latest_attendance_date = group_attendance['date'].max()
                else:
                    latest_attendance_date = pd.Timestamp('1900-01-01')  # Default old date for new admissions groups
            else:
                latest_attendance_date = pd.Timestamp('1900-01-01')  # Default old date for new admissions groups
            print(latest_attendance_date)
            engagements = engagement_df[engagement_df['admissions-group-name'] == admissions_group_name]
            
            for _, engagement in engagements.iterrows():
                if engagement['engagement-date'] > latest_attendance_date:
                    new_row = create_attendance_row(engagement)
                    new_attendance_records.append(new_row)

        new_attendance_df = pd.DataFrame(new_attendance_records)
        """
        # Post new attendance records if any
        if not new_attendance_df.empty:
            new_attendance_df['date'] = new_attendance_df['date'].dt.strftime('%m/%d/%Y') + " 05:30 AM"
            for _, record in new_attendance_df.iterrows():
                admissions_group_name = record['admissions-group-name']
                print(f"Posting attendance for admissions group: {admissions_group_name}")
                url = f"{ums_base_url}/Attendance"
                await post_attendance_record(session, url, ums_headers, record.to_dict())
        else:
            print("No new attendance records to process.")
        
        """
        
        print(new_attendance_df.head())
        return new_attendance_df


await main()



Fetching Student data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 0
Fetched 100 new records, Total fetched: 100
Fetching Student data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 100
Fetched 100 new records, Total fetched: 200
Fetching Student data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 200
Fetched 100 new records, Total fetched: 300
Fetching Student data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 300
Fetched 100 new records, Total fetched: 400
Fetching Student data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 400
Fetched 100 new records, Total fetched: 500
Fetching Student data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 500
Fetched 100 new records, Total fetched: 600
Fetching Student data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 600
Fetched 100 new records, Total fetched: 700
Fetching Student data from https://app.edoofa.com/version-test/a

Fetched 100 new records, Total fetched: 5100
Fetching Attendance data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 5100
Fetched 100 new records, Total fetched: 5200
Fetching Attendance data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 5200
Fetched 100 new records, Total fetched: 5300
Fetching Attendance data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 5300
Fetched 100 new records, Total fetched: 5400
Fetching Attendance data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 5400
Fetched 100 new records, Total fetched: 5500
Fetching Attendance data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 5500
Fetched 34 new records, Total fetched: 5534
Exiting loop, fetched less than 100 records.
Fetched 5534 records for Attendance.
Fetching Student data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 0
Fetched 100 new records, Total fetched: 100
Fetching Student data from https://edoofa-porta

Fetched 100 new records, Total fetched: 1500
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1500
Fetched 100 new records, Total fetched: 1600
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1600
Fetched 100 new records, Total fetched: 1700
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1700
Fetched 100 new records, Total fetched: 1800
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1800
Fetched 100 new records, Total fetched: 1900
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1900
Fetched 100 new records, Total fetched: 2000
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 2000
Fetched 32 new records, Total fetched: 2032
Exiting loop, fetched less than 100 records.
Fetched 2032 records for Engagement.
Fetching Engagement data from https://ed

Fetched 100 new records, Total fetched: 1200
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1200
Fetched 100 new records, Total fetched: 1300
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1300
Fetched 100 new records, Total fetched: 1400
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1400
Fetched 100 new records, Total fetched: 1500
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1500
Fetched 100 new records, Total fetched: 1600
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1600
Fetched 100 new records, Total fetched: 1700
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1700
Fetched 100 new records, Total fetched: 1800
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1800
Fetched 100 new records, To

2024-01-20
Processing admissions group: Kuzivakwashe 2K23FEB1895
2023-12-13
Processing admissions group: LISFORD 2K22DEC1308
2024-01-20
Processing admissions group: Lorraine 2K23JAN0518
2024-01-20
Processing admissions group: Maggie 2K21APR2662
1900-01-01 00:00:00
Processing admissions group: Abigail 2K22DEC2057
2024-01-20
Processing admissions group: Makanaka 2K23MAY1103
2024-01-03
Processing admissions group: Makanakaishe 2K21MAR2256
1900-01-01 00:00:00
Processing admissions group: Kudzanai 2K21APR2637
1900-01-01 00:00:00
Processing admissions group: Nicole 2K22FEB1010
1900-01-01 00:00:00
Processing admissions group: Vimbayi 2K22OCT1490
2024-01-03
Processing admissions group: Panashe 2K21JUL1642
1900-01-01 00:00:00
Processing admissions group: Martin 2K21JUL1410
1900-01-01 00:00:00
Processing admissions group: Mary 2K21MAY1750
1900-01-01 00:00:00
Processing admissions group: MASIKA 2K21SEP0430
1900-01-01 00:00:00
Processing admissions group: Miriam 2K22FEB0209
2023-09-02
Processing a

2024-01-03
Processing admissions group: Egnes 2K21SEP0790
1900-01-01 00:00:00
Processing admissions group: Amarachkwu 2K21JUL2129
2024-01-06
Processing admissions group: Elton 2K23JAN0919
2024-01-03
Processing admissions group: Eugenia 2K21FEB2538
1900-01-01 00:00:00
Processing admissions group: Fanuel 2K22NOV1104
2024-01-06
Processing admissions group: Melody 2K20SEP3061
1900-01-01 00:00:00
Processing admissions group: Fungisai 2K22JUN2116
1900-01-01 00:00:00
Processing admissions group: Gelean 2K22JAN1857
1900-01-01 00:00:00
Processing admissions group: Gervais 2K23JAN1002
2024-01-17
Processing admissions group: Gracious 2K22JUL1545
2024-01-06
Processing admissions group: Hazel 2K23JAN0642
2024-01-03
Processing admissions group: Heather 2K20AUG1904
1900-01-01 00:00:00
Processing admissions group: Heggah 2K23FEB0452
2024-01-03
Processing admissions group: Hester 2K23FEB1331
2024-01-06
Processing admissions group: Hilario 2K23FEB0134
2024-01-03
Processing admissions group: Ivy 2K23FEB1

2024-01-03
Processing admissions group: Nicole 2K21JAN0875
2024-01-17
Processing admissions group: Nqoba 2K23MAR1711
2024-01-24
Processing admissions group: Pauline 2K23MAR0923
2024-01-17
Processing admissions group: Primrose 2K21SEP0156
1900-01-01 00:00:00
Processing admissions group: Princess 2K22AUG0836
2024-01-03
Processing admissions group: Priscilla 2K21NOV0104
2024-01-20
Processing admissions group: Queen 2K21NOV1078
1900-01-01 00:00:00
Processing admissions group: Reward 2K22APR2378
2024-01-17
Processing admissions group: Ruvimbo 2K22APR2648
1900-01-01 00:00:00
Processing admissions group: Sandra 2K21NOV0228
2024-01-20
Processing admissions group: Shalom 2k22DEC0848
2024-01-17
Processing admissions group: Sibongile 2K23MAR0635
2024-01-17
Processing admissions group: Sikhululekile 2K23MAR2935
2024-01-17
Processing admissions group: Sithabisiwe 2K23JAN1904
2024-01-20
Processing admissions group: Susan 2K23MAY2338
2024-01-20
Processing admissions group: Tadiwanashe 2K22JUN1129
202

Processing admissions group: Blessing 2K23FEB2334
2024-01-17
Processing admissions group: Brenda 2K23FEB0992
2024-01-17
Processing admissions group: Ashley 2K22MAR1652
1900-01-01 00:00:00
Processing admissions group: Rutendo 2K22MAR1777
2024-01-20
Processing admissions group: Natashe 2K22MAR1410
1900-01-01 00:00:00
Processing admissions group: Rufaro 2K23FEB0692
2024-01-06
Processing admissions group: Allen 2K22SEP2695
2024-01-17
Processing admissions group: Neo 2K23MAY0223
2024-01-03
Processing admissions group: Wilma 2K21SEP0848
2023-12-23
Processing admissions group: Bhekinkosi 2K22OCT2498
2024-01-17
Processing admissions group: Natasha 2K22JAN1001
1900-01-01 00:00:00
Processing admissions group: Courage 2K22JUL1682
2023-12-16
Processing admissions group: Ruth 2K22DEC1240
2024-01-20
Processing admissions group: Ownwell 2K20AUG2760
1900-01-01 00:00:00
Processing admissions group: Charleen 2K23JAN0501
2023-12-16
Processing admissions group: Michael 2K23FEB0902
2024-01-03
Processing ad

""


In [14]:
import nest_asyncio
nest_asyncio.apply()

import aiohttp
import asyncio
import json
import pandas as pd
from datetime import datetime 
from datetime import timedelta


student_df = None
engagement_df = None
ums_att_df = None
ums_avg_att_df = None
ums_upload_avg_att=None

edoofy_base_url = "https://edoofa-portal.bubbleapps.io/api/1.1/obj"
edoofy_bearer_token = "2cde31d8f48919a2db1467cc06a56132"
edoofy_headers = {'Authorization': f'Bearer {edoofy_bearer_token}'}

ums_base_url = "https://app.edoofa.com/version-test/api/1.1/obj"
ums_bearer_token = "786720e8eb68de7054d1149b56cc04f9"
ums_headers = {'Authorization': f'Bearer {ums_bearer_token}'}


async def fetch_table_data(session, base_url, headers, table, constraints=None):
    records = []
    cursor = 0
    total_fetched = 0

    while True:
        params = {'limit': 100, 'cursor': cursor}
        if constraints:
            params['constraints'] = json.dumps(constraints)

        api_url = f"{base_url}/{table}"
        print(f"Fetching {table} data from {base_url}... Cursor: {cursor}")

        async with session.get(api_url, headers=headers, params=params) as response:
            if response.status != 200:
                print(f"Failed to fetch data from {table}: {await response.text()}")
                break

            data = await response.json()
            new_records = data['response']['results']
            records.extend(new_records)
            total_fetched += len(new_records)

            print(f"Fetched {len(new_records)} new records, Total fetched: {total_fetched}")

            cursor += 100

            if len(new_records) < 100:
                print(f"Exiting loop, fetched less than 100 records.")
                break

    df = pd.DataFrame(records)
    print(f"Fetched {len(df)} records for {table}.")
    return df



def create_attendance_row(engagement_record):
    attendance_row = {
        'admissions-group-name': engagement_record['admissions-group-name'],  
        'attendance-type': engagement_record['engagement-type'],  
        'date': engagement_record['adjusted-engagement-date'],  
        'ewyl-group-name': engagement_record['ewyl-group-name'],  
        'present': engagement_record['daily-attendance'],  
    }

    return attendance_row

async def post_attendance_record(session, url, headers, record):
    try:
        async with session.post(url, headers=headers, json=record) as response:
            if response.status == 200:
                print("Record posted successfully.")
                return await response.json()
            else:
                print(f"Failed to post record: {await response.text()}")
                return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


async def main():
    async with aiohttp.ClientSession() as session:
        ums_student_df = await fetch_table_data(session, ums_base_url, ums_headers, "Student")
        ums_att_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance")  
        student_constraints = [{'key': 'indian-edoofian', 'constraint_type': 'equals', 'value': 'yes'}]
        student_df = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Student", constraints=student_constraints)
        
        engagement_df = pd.DataFrame()
        for engagement_type in ['IE Call', 'IE Chat', 'Activity', 'Lesson']:
            engagement_data = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Engagement", [{'key': 'engagement-type', 'constraint_type': 'equals', 'value': engagement_type}])
            engagement_df = pd.concat([engagement_df, engagement_data], ignore_index=True)
        
        engagement_df = pd.merge(engagement_df, student_df[['_id', 'KAM-group-name', 'EWYL-group-name']], left_on='student', right_on='_id', how='left')
        engagement_df.rename(columns={'KAM-group-name': 'admissions-group-name', 'EWYL-group-name': 'ewyl-group-name', '_id_x': '_id'}, inplace=True)
        engagement_df.drop(['_id_y'], axis=1, inplace=True)
        engagement_df.dropna(subset=['admissions-group-name'], inplace=True)
        
        # Convert dates to datetime objects for accurate comparisons
        ums_att_df['date'] = pd.to_datetime(ums_att_df['date']).dt.tz_localize(None)
        engagement_df['engagement-date'] = pd.to_datetime(engagement_df['engagement-date']).dt.tz_localize(None)

        engagement_df['adjusted-engagement-date'] = pd.to_datetime(engagement_df['engagement-date']) + timedelta(hours=5, minutes=30)
        engagement_df.to_csv("engagement_df.csv", index=False)
        new_attendance_records = []

        for _, student in student_df.iterrows():
            admissions_group_name = student['KAM-group-name']
            print(f"Processing admissions group: {admissions_group_name}")

            # Get the latest engagement date for the current student
            latest_engagement_date = engagement_df[engagement_df['admissions-group-name'] == admissions_group_name]['adjusted-engagement-date'].max()

            # Get the latest UMS attendance date for the current student
            latest_ums_attendance_date = ums_att_df[ums_att_df['admissions-group-name'] == admissions_group_name]['date'].max()

            # Process only if the latest engagement date is after the latest UMS attendance date
            if pd.isnull(latest_engagement_date) or pd.isnull(latest_ums_attendance_date) or latest_engagement_date <= latest_ums_attendance_date:
                # Skip this student if there's no latest engagement date or the latest engagement date is not after the latest UMS attendance date
                continue

            # Filter engagements that occurred after the latest UMS attendance date
            engagements_to_process = engagement_df[(engagement_df['admissions-group-name'] == admissions_group_name) & 
                                                   (engagement_df['adjusted-engagement-date'] > latest_ums_attendance_date)]

            # Process each relevant engagement
            for _, engagement in engagements_to_process.iterrows():
                new_row = create_attendance_row(engagement)
                new_attendance_records.append(new_row)

        # Convert the list of new rows to a DataFrame
        new_attendance_df = pd.DataFrame(new_attendance_records)

        """
        # Post new attendance records if any
        if not new_attendance_df.empty:
            new_attendance_df['date'] = new_attendance_df['date'].dt.strftime('%m/%d/%Y') + " 05:30 AM"
            for _, record in new_attendance_df.iterrows():
                admissions_group_name = record['admissions-group-name']
                print(f"Posting attendance for admissions group: {admissions_group_name}")
                url = f"{ums_base_url}/Attendance"
                await post_attendance_record(session, url, ums_headers, record.to_dict())
        else:
            print("No new attendance records to process.")
        """
        
        print(new_attendance_df.head())
        return new_attendance_df
        new_attendance_df.to_csv("new_attendance_df.csv", index= False)

await main()



Fetching Student data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 0
Fetched 100 new records, Total fetched: 100
Fetching Student data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 100
Fetched 100 new records, Total fetched: 200
Fetching Student data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 200
Fetched 100 new records, Total fetched: 300
Fetching Student data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 300
Fetched 100 new records, Total fetched: 400
Fetching Student data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 400
Fetched 100 new records, Total fetched: 500
Fetching Student data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 500
Fetched 100 new records, Total fetched: 600
Fetching Student data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 600
Fetched 100 new records, Total fetched: 700
Fetching Student data from https://app.edoofa.com/version-test/a

Fetched 100 new records, Total fetched: 1100
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1100
Fetched 100 new records, Total fetched: 1200
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1200
Fetched 100 new records, Total fetched: 1300
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1300
Fetched 100 new records, Total fetched: 1400
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1400
Fetched 100 new records, Total fetched: 1500
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1500
Fetched 100 new records, Total fetched: 1600
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1600
Fetched 100 new records, Total fetched: 1700
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1700
Fetched 100 new records, To

Fetched 100 new records, Total fetched: 800
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 800
Fetched 100 new records, Total fetched: 900
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 900
Fetched 100 new records, Total fetched: 1000
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1000
Fetched 100 new records, Total fetched: 1100
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1100
Fetched 100 new records, Total fetched: 1200
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1200
Fetched 100 new records, Total fetched: 1300
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1300
Fetched 100 new records, Total fetched: 1400
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1400
Fetched 100 new records, Total 

Processing admissions group: KUDAKWASHE 2K23FEB1701
Processing admissions group: AUSTIN 2K21SEP0418
Processing admissions group: Kudziwaishe 2K22NOV0590
Processing admissions group: Linda 2K23JAN2363
Processing admissions group: Carren 2K21NOV1275
Processing admissions group: Miriam 2K22MAY1702
Processing admissions group: Tafadzwa 2K23JAN2336
Processing admissions group: Moreblessing 2K22AUG1328
Processing admissions group: Patron 2K21JUN0812
Processing admissions group: Precious 2K22AUG2211
Processing admissions group: Cleopatra 2K23JAN2354
Processing admissions group: Redeem 2K22SEP1909
Processing admissions group: Shammah 2K23JAN2795
Processing admissions group: Shyne 2k21OCT0950
Processing admissions group: Talent 2K22APR1238
Processing admissions group: Thubelihle 2K23FEB1715
Processing admissions group: Tinevimbo 2K21DEC1757
Processing admissions group: Tino 2K22OCT2525
Processing admissions group: Tracy 2K22FEB1632
Processing admissions group: Vincient 2K23JAN2402
Processing ad

Processing admissions group: Justine 2K21SEP3082
Processing admissions group: Sekwila 2K23MAR2287
Processing admissions group: Kimberly 2K23MAY1595
Processing admissions group: King 2K21APR0942
Processing admissions group: Kudakwashe 2K22JUN2309
Processing admissions group: Kudzai 2K21NOV1662
Processing admissions group: Kudzai 2K21OCT2805
Processing admissions group: Kudzanai 2K22OCT1748
Processing admissions group: Learnfirst 2K22JUN1753
Processing admissions group: Lerato 2K22JAN1734
Processing admissions group: Leslie 2K21DEC2023
Processing admissions group: Linnety 2K21OCT2204
Processing admissions group: Loice 2K22OCT1152
Processing admissions group: Lorraine 2K21NOV2505
Processing admissions group: Lydia 2K23APR1845
Processing admissions group: Vimbikai 2K22MAY0732
Processing admissions group: Manwere 2K21NOV1306
Processing admissions group: Margret 2K21MAR0141
Processing admissions group: Marlon 2K23FEB1490
Processing admissions group: Marvellous 2K22AUG0965
Processing admissio

Processing admissions group: Theresa 2K21NOV1352
Processing admissions group: Bonface 2K21NOV1783
Processing admissions group: Changadzo 2K21NOV2953
Processing admissions group: Fambai 2K21DEC1153
Processing admissions group: Kelvin 2K22JAN1985
Processing admissions group: Patience 2K22JAN1966R
Processing admissions group: Lisben 2K22JAN1892
Processing admissions group: Asher 2K22JAN2203
Processing admissions group: Elshebba 2K22MAR0216
Processing admissions group: Lorraine 2K22MAR2210
Processing admissions group: Tatenda 2K22MAR2190
Processing admissions group: Shalom 2K22MAR2556
Processing admissions group: Munashe 2K22MAR2142
Processing admissions group: LISA 2K22MAR2589
Processing admissions group: Thamsanqa 2K22MAR2994
Processing admissions group: Albert 2K22MAR3194
Processing admissions group: Sally 2K22APR0722
Processing admissions group: Melrose 2K22APR1652
Processing admissions group: Beverley 2K22APR1912
Processing admissions group: Natalie 2K22APR1572
Processing admissions g

Processing admissions group: Tadiwanashe 2K20SEP134
Processing admissions group: Ryan 2K20SEP0101
Processing admissions group: Munyaradzi 2K20SEP1107
Processing admissions group: Silence 2K20SEP1649
Processing admissions group: Wandile 2K20OCT0643
Processing admissions group: Sandra 2K20OCT0941
Processing admissions group: Brian 2K20NOV1120
Processing admissions group: Thrive 2K20DEC1422
Processing admissions group: Tendai 2K21JAN0827
Processing admissions group: Zibusiso 2K21FEB0909
Processing admissions group: Ropafadzo 2K21FEB2050
Processing admissions group: Cynthia 2K21MAR0258
Processing admissions group: Rufaro 2K21MAR0577
Processing admissions group: Arnold 2K21MAR1017
Processing admissions group: Anesu 2K21MAR2043
Processing admissions group: Christine 2K21MAR2039
Processing admissions group: Rumbidzai 2K21MAR2640
Processing admissions group: Joyleen 2K21APR0268
Processing admissions group: Gladsome 2K21MAR3101
Processing admissions group: Wesley 2K21APR1367
Processing admissio

""
